## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df = city_data_df.drop(columns='Unnamed: 0')
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,mahebourg,-20.4081,57.7000,72.25,83,19,8.86,MU,few clouds
1,namibe,-15.1961,12.1522,71.80,65,16,3.13,AO,few clouds
2,nanortalik,60.1432,-45.2371,38.64,83,19,19.37,GL,few clouds
3,vaini,-21.2000,-175.2000,69.96,94,90,6.91,TO,light rain
4,shache,38.4167,77.2406,76.50,27,100,7.38,CN,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Use a conditional set to false to control a while loop
conditional = False

# Loop through temperature selection until both input values can be converted to floats
while conditional == False:
    input_1 = input('What is the upper limit of your desired vacation temperature range?  ')
    input_2 = input('How about the lower limit of temperature?  ')
# Try - except statement to catch errors attempting to convert a string to a float    
    try:
        temp_range_high = float(input_1)
        temp_range_low = float(input_2)
# If loop make sure that the higher of the entered temperatures is the top of the range        
        if temp_range_high <= temp_range_low:
            pass_variable = temp_range_high
            temp_range_high = temp_range_low
            temp_range_low = pass_variable
        print(f'You want a vacation with max temperature between {temp_range_low:.1f} and {temp_range_high:.1f}')
        response = input('is that right?  y, or n')
# If statement checks to make sure that the entered temperature range is as requested        
        if (response == 'y'):
            conditional = True
        else:
            conditional = False
    except:
        print('hmm, it looks like you entered a string for one of your temperatures')
        print('please enter your selected temperatures again')

What is the upper limit of your desired vacation temperature range?  50
How about the lower limit of temperature?  100
You want a vacation with max temperature between 50.0 and 100.0
is that right?  y, or nn
What is the upper limit of your desired vacation temperature range?  80
How about the lower limit of temperature?  70
You want a vacation with max temperature between 70.0 and 80.0
is that right?  y, or ny


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
selected_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= temp_range_high) & (city_data_df['Max Temp'] >= temp_range_low)]
selected_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,mahebourg,-20.4081,57.7000,72.25,83,19,8.86,MU,few clouds
1,namibe,-15.1961,12.1522,71.80,65,16,3.13,AO,few clouds
4,shache,38.4167,77.2406,76.50,27,100,7.38,CN,overcast clouds
25,puerto ayora,-0.7393,-90.3518,75.54,84,18,8.01,EC,few clouds
29,otukpo,7.1913,8.1329,78.89,81,50,8.72,NG,scattered clouds


In [13]:
# 4a. Determine if there are any empty rows.
selected_cities_df.isnull().sum()

City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = selected_cities_df.copy()
clean_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,mahebourg,-20.4081,57.7000,72.25,83,19,8.86,MU,few clouds
1,namibe,-15.1961,12.1522,71.80,65,16,3.13,AO,few clouds
4,shache,38.4167,77.2406,76.50,27,100,7.38,CN,overcast clouds
25,puerto ayora,-0.7393,-90.3518,75.54,84,18,8.01,EC,few clouds
29,otukpo,7.1913,8.1329,78.89,81,50,8.72,NG,scattered clouds
34,virden,39.5009,-89.7679,71.10,45,76,3.87,US,broken clouds
45,rikitea,-23.1203,-134.9692,75.11,70,97,12.41,PF,overcast clouds
47,vanimo,-2.6741,141.3028,78.53,89,98,6.13,PG,overcast clouds
48,caravelas,-17.7125,-39.2481,75.07,86,9,6.29,BR,clear sky
51,sao filipe,14.8961,-24.4956,74.16,75,86,6.85,CV,overcast clouds


In [32]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mahebourg,MU,72.25,few clouds,-20.4081,57.7000,
1,namibe,AO,71.80,few clouds,-15.1961,12.1522,
4,shache,CN,76.50,overcast clouds,38.4167,77.2406,
25,puerto ayora,EC,75.54,few clouds,-0.7393,-90.3518,
29,otukpo,NG,78.89,scattered clouds,7.1913,8.1329,
34,virden,US,71.10,broken clouds,39.5009,-89.7679,
45,rikitea,PF,75.11,overcast clouds,-23.1203,-134.9692,
47,vanimo,PG,78.53,overcast clouds,-2.6741,141.3028,
48,caravelas,BR,75.07,clear sky,-17.7125,-39.2481,
51,sao filipe,CV,74.16,overcast clouds,14.8961,-24.4956,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# # 6b. Iterate through the hotel DataFrame.
# for index, row in hotel_df.iterrows():
#     latitude = row['Lat']
#     longitude = row['Lng']
#     location = f'{latitude},{longitude}'
   
#     params['location'] = location
 

#         # 6c. Get latitude and longitude from DataFrame


#         # 6d. Set up the base URL for the Google Directions API to get JSON data.
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     hotel_output = requests.get(base_url,params = params).json()
   

#     # 6e. Make request and retrieve the JSON data from the search. 
   
    
#     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    location = f'{latitude},{longitude}'
   
    params['location'] = location
 

        # 6c. Get latitude and longitude from DataFrame


        # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_output = requests.get(base_url,params = params).json()
    try:
        hotel_df.loc[index,['Hotel Name']] = hotel_output['results'][0]['name']
    except:
        No_hotel_city = row['City']
        print(f'No hotel found within 5,000 m of {No_hotel_city}')
        pass
    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

No hotel found within 5,000 m of poya
No hotel found within 5,000 m of gamba
No hotel found within 5,000 m of aketi
No hotel found within 5,000 m of marzuq
No hotel found within 5,000 m of panguna
No hotel found within 5,000 m of nioro
No hotel found within 5,000 m of guisa
No hotel found within 5,000 m of ndende
No hotel found within 5,000 m of caxito


In [84]:
# 7. Drop the rows where there is no Hotel Name.
null_rows = hotel_df.loc[hotel_df['Hotel Name'] == ''].index
for index in null_rows:
    hotel_df.drop(index = index,inplace=True)
clean_hotel_df = hotel_df.copy()
hotel_df.loc[hotel_df['Hotel Name'] == '']

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [85]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [86]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt/><dd>{City}</dd>
<dt>Country</dt/><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt/><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [87]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))